<a href="https://colab.research.google.com/github/eggtartplus/nlp/blob/main/dl_ass1EX1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check nvcc version
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
# Check GCC version
!gcc --version

gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [3]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.13.1+cu116
True


In [4]:
# Install mmcv
!pip install mmcv -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.1/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.1/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 KB 24.7 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-1.7.1-py2.py3-none-any.whl size=930735 sha256=6531497fd234148425744ce30bb1e4f8f32cc64817d4c45b8cb24e35214f8426
  Stored in directory: /root/.cache/pip/wheels/ae/69/b3/d097f4d265ae854179c4263f446283e3f402caa0dbf0083d05
Successfully built mmcv


In [5]:
# Clone mmcls repository
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/

# Install MMClassification from source
!pip install -e . 

Cloning into 'mmclassification'...
remote: Enumerating objects: 14232, done.
remote: Counting objects: 100% (806/806), done.
remote: Compressing objects: 100% (475/475), done.
remote: Total 14232 (delta 416), reused 600 (delta 327), pack-reused 13426
Receiving objects: 100% (14232/14232), 12.34 MiB | 13.48 MiB/s, done.
Resolving deltas: 100% (9761/9761), done.
/content/mmclassification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmclassification
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmcls


In [6]:
# Check MMClassification installation
import mmcls
print(mmcls.__version__)

0.25.0


/usr/local/lib/python3.9/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [7]:
# Confirm the config file exists
!ls configs/resnet/resnet101_8xb32_in1k.py

# Specify the path of the config file and checkpoint file.
config_file = 'configs/resnet/resnet101_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet101_batch256_imagenet_20200708-753f3608.pth'

configs/resnet/resnet101_8xb32_in1k.py


In [8]:
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot

# Specify the device, if you cannot use GPU, you can also use CPU 
# by specifying `device='cpu'`.
device = 'cuda:0'
# device = 'cpu'

# Build the model according to the config file and load the checkpoint.
model = init_model(config_file, checkpoint_file, device=device)

/usr/local/lib/python3.9/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet101_batch256_imagenet_20200708-753f3608.pth


Downloading: "https://download.openmmlab.com/mmclassification/v0/resnet/resnet101_batch256_imagenet_20200708-753f3608.pth" to /root/.cache/torch/hub/checkpoints/resnet101_batch256_imagenet_20200708-753f3608.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

/content/mmclassification/mmcls/apis/inference.py:45: UserWarning: Class names are not saved in the checkpoint's meta data, use imagenet by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [9]:
# The model's inheritance relationship
model.__class__.__mro__

(mmcls.models.classifiers.image.ImageClassifier,
 mmcls.models.classifiers.base.BaseClassifier,
 mmcv.runner.base_module.BaseModule,
 torch.nn.modules.module.Module,
 object)

In [10]:
# mount google drive for dataset access
from google.colab import drive
drive.mount('/content/drive')
import os
!cd '/content/drive/My Drive/Colab Notebooks/mmclassification' 
#!mkdir mmclassification
os.chdir("/content/drive/My Drive/Colab Notebooks/mmclassification")
     

Mounted at /content/drive


In [11]:
!pwd

/content/drive/My Drive/Colab Notebooks/mmclassification


In [14]:
%%writefile configs/resnet/resnet101_8xb32_in1k.py
_base_ = [
    '../_base_/models/resnet101.py', '../_base_/datasets/imagenet_bs32.py',
    '../_base_/schedules/imagenet_bs256.py', '../_base_/default_runtime.py'
]

model = dict(
    backbone=dict(
        init_cfg = dict(
            type='Pretrained', 
            checkpoint='https://download.openmmlab.com/mmclassification/v0/resnet/resnet101_batch256_imagenet_20200708-753f3608.pth', 
            prefix='backbone')
    ),
    head=dict(
        num_classes=5,
        topk = (1, )
    ))

dataset_type = 'CustomDataset'

data = dict(
    train=dict(type=dataset_type, data_prefix="data/flower_dataset/training_set"),
    val=dict(type=dataset_type, data_prefix="data/flower_dataset/val_set", ann_file=None),
)

# Modify the evaluation metric
evaluation = dict(interval=1, metric_options=dict(topk=(1,)))
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)

# Specify the learning rate scheduler
lr_config = dict(policy='step', step=[15])
runner = dict(type='EpochBasedRunner', max_epochs=20)

# # Specify the work directory
# work_dir = './work_dirs/flower_dataset'

# Output logs
checkpoint_config = dict(interval=10)
log_config = dict(interval=10)


Overwriting configs/resnet/resnet101_8xb32_in1k.py


In [15]:
#%cd /content
#!cd '/content/drive/MyDrive/Colab Notebooks/mmclassification'
# run the code
!python tools/train.py configs/resnet/resnet101_8xb32_in1k.py

/usr/local/lib/python3.9/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
/content/mmclassification/mmcls/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application a